Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.  
Copyright (c) 2017, Pytorch contributors All rights reserved.
## BSD 3-Clause License
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# ResNet50 for PyTorch with GPU Migration

In this notebook we will demonstrate ResNet50 model which is based on open source implementation of ResNet50. It has been enabled using an experimental feature called GPU migration toolkit and it can be trained using Pytorch on 8 HPUs.

#### Clone the Model-References repository

In [1]:
!git clone https://github.com/habanaai/Model-References

Cloning into 'Model-References'...
remote: Enumerating objects: 18770, done.
remote: Counting objects: 100% (4559/4559), done.
remote: Compressing objects: 100% (2021/2021), done.
remote: Total 18770 (delta 2435), reused 4301 (delta 2238), pack-reused 14211
Receiving objects: 100% (18770/18770), 119.22 MiB | 26.76 MiB/s, done.
Resolving deltas: 100% (10121/10121), done.


#### Set the ENV variables

In [2]:
%set_env PYTHONPATH=/root/Gaudi2-Workshop/Model-Migration/Model-References:/usr/lib/habanalabs/:/root
%set_env PYTHON=/usr/bin/python3.8

env: PYTHONPATH=/root/Gaudi2-Workshop/Model-Migration/Model-References:/usr/lib/habanalabs/:/root
env: PYTHON=/usr/bin/python3.8


#### Naviagte to the model to begin the run

In [3]:
%cd /root/Gaudi2-Workshop/Model-Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision

/root/Gaudi2-Workshop/Model-Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Download dataset
ImageNet 2012 dataset needs to be organized according to PyTorch requirements, and as specified in the scripts of [imagenet-multiGPU.torch](https://github.com/soumith/imagenet-multiGPU.torch).

#### Import GPU Migration Toolkit package and Habana Torch Library
Look into train.py

In [4]:
%%sh
cat -n train.py | head -n 20 | tail -n 17

     4	import habana_frameworks.torch.gpu_migration
     5	import datetime
     6	import os
     7	import time
     8	import warnings
     9	
    10	import presets
    11	import torch
    12	import torch.utils.data
    13	import torchvision
    14	import transforms
    15	import utils
    16	import habana_frameworks.torch.utils.experimental as htexp
    17	from sampler import RASampler
    18	from torch import nn
    19	from torch.utils.data.dataloader import default_collate
    20	from torchvision.transforms.functional import InterpolationMode


#### Placing mark_step()

In [5]:
%%sh
cat -n train.py | head -n 51 | tail -n 14

    38	        optimizer.zero_grad(set_to_none=True)
    39	        if scaler is not None:
    40	            scaler.scale(loss).backward()
    41	            htcore.mark_step()
    42	            if args.clip_grad_norm is not None:
    43	                # we should unscale the gradients of optimizer's assigned params if do gradient clipping
    44	                scaler.unscale_(optimizer)
    45	                nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
    46	            scaler.step(optimizer)
    47	            htcore.mark_step()
    48	            scaler.update()
    49	        else:
    50	            loss.backward()
    51	            htcore.mark_step()


#### Run the following command to start multi-HPU training.

```bash
GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 8 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path=/root/software/data/pytorch/imagenet/ILSVRC2012 --workers=8 --epochs=1 --opt=sgd --amp
```

In [6]:
!GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 8 train.py --batch-size=256 --model=resnet50 --device=cuda \
--data-path=/root/data/huggingface/datasets/imagenet/ILSVRC2012 \
--workers=8 --epochs=1 --opt=sgd --amp

[2024-07-17 05:49:48,504] torch.distributed.run: [WARNING] 
[2024-07-17 05:49:48,504] torch.distributed.run: [WARNING] *****************************************
[2024-07-17 05:49:48,504] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-07-17 05:49:48,504] torch.distributed.run: [WARNING] *****************************************
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/gpu_migration/__init__.py:46: UserWarning: apex not installed, gpu_migration will not swap api for this package.
  warnings.warn(
gpu migration log will be saved to /var/log/habana_logs/gpu_migration_logs/2024-07-17/05-49-51/gpu_migration_700.log
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/gpu_migration/__init__.py:46: UserWarning: apex not installed, gpu_migration will not swap api

[2023-06-09 23:17:26] train.py:32
    [context]:         image, target = image.to(device, non_blocking=True), target.to(device, non_blocking=True)

    [hpu_match]: torch.Tensor.to(args=(device(type='cuda'),), kwargs={'non_blocking': True}, ) --> torch.Tensor.to(args=('hpu',), kwargs={non_blocking=True, })

[2023-06-09 23:17:26] train.py:33
    [context]:         with torch.cuda.amp.autocast(enabled=scaler is not None):

    [hpu_match]: torch.autocast.__init__(device_type=cuda, dtype=torch.float16, enabled=True, cache_enabled=True, ) --> torch.autocast.__init__(device_type=hpu, dtype=None, enabled=True, cache_enabled=True, )

[2023-06-09 23:17:26] /usr/local/lib/python3.8/dist-packages/torch/cuda/amp/common.py:7
    [context]:     return not (torch.cuda.is_available() or find_spec('torch_xla'))

    [hpu_match]: torch.cuda.is_available() --> torch.hpu.is_available()

[2023-06-09 23:17:37] /root/tf/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvisi

Epoch: [0]  [370/622]  eta: 0:00:35  lr: 0.1  img/s: 4708.559641186386  loss: 6.0441 (6.4522)  acc1: 1.9531 (1.2747)  acc5: 6.2500 (4.5539)  time: 0.0542  data: 0.0080  max mem: 16966
Epoch: [0]  [380/622]  eta: 0:00:33  lr: 0.1  img/s: 3969.727520264712  loss: 6.0329 (6.4277)  acc1: 1.9531 (1.3822)  acc5: 6.6406 (4.7676)  time: 0.0568  data: 0.0114  max mem: 16966
Epoch: [0]  [390/622]  eta: 0:00:31  lr: 0.1  img/s: 5295.522353315996  loss: 5.9631 (6.4052)  acc1: 1.9531 (1.4258)  acc5: 7.8125 (5.0781)  time: 0.0538  data: 0.0095  max mem: 16966
Epoch: [0]  [400/622]  eta: 0:00:29  lr: 0.1  img/s: 3096.166358897938  loss: 5.9305 (6.3836)  acc1: 1.9531 (1.4863)  acc5: 8.2031 (5.3163)  time: 0.0628  data: 0.0150  max mem: 16966
Epoch: [0]  [410/622]  eta: 0:00:28  lr: 0.1  img/s: 2408.498940252397  loss: 5.9192 (6.3629)  acc1: 2.7344 (1.5532)  acc5: 9.3750 (5.4874)  time: 0.0918  data: 0.0395  max mem: 16966
Epoch: [0]  [420/622]  eta: 0:00:26  lr: 0.1  img/s: 3030.912933452077  loss: 5.

Training time 0:01:23
